LDA (Latent Dirichlet Allocation)

LDA - частный случай подхода pLSA.

Название из-за того, что в качестве априорного распределения используется распределение Дирихле.

Рассмотрим отдельно несколько тем, которые нужны для понимания метода LDA

## Plate Notation

Plate Notation - способ схематичного изображения графических веротяностных моделей, которые содержат большое число повторяющихся элементов.

Цель: удобство восприятия модели

Как читается Plate Notation?
- Стрелка - наличие зависимости
- Закарашенные круги - наблюдаемые переменные
- Пустые - латентные переменные
- Прямоугольник с числом N внизу - означает "повторяется N раз"

Проиллюстрируем на примере pLSA

<img src="img/plate0.png" width=300>



У нас есть M документов (внешняя плашка) и в каждом таком документе N слов (внутренняя). 

С каждым словом связана латентная переменная c, обозначающая тематику. Веротяность тематики зависит от документа: P(c|d). 

А распределение конкретного слова w зависит от выбранной тематики P(w|c).

Задача pLSA: найти конкретные распределния P(c|d) и P(w|c).

-----

## Dirichlet Distribution

Распределение Дирихле  - вероятностное распределение k-элементного вектора $x=(x_1,x_2 \dots x_k)$ при условии $\sum{x_i}=1$.

Множество точек, где $\sum{x_i}=1$ это <a href="https://ru.wikipedia.org/wiki/%D0%A1%D0%B8%D0%BC%D0%BF%D0%BB%D0%B5%D0%BA%D1%81">симплекс</a>. Таким образом, РД - это распределение вероятности k-элементной комбинации на симплексе.

По сути, это веротяностное распределение дискретного распределения :)

### Параметры рапсределения

У распределения Дирихле есть параметры $\hat{\alpha} = (\alpha_1, \alpha_2, \dots \alpha_k)$, задающие конкретный вид распределения: $\alpha_i > 0$.

Посмотрим, как меняется распределение при разных параметрах. Для наглядности возьмем k=3.

|$\bar{\alpha}$|распределение|
|---|---|
|(1,1,1) | равномерное распределение |
|(0.2,0.2,0.2) | больше вероятности у краев (когда часть $x_i$ зануляется)|
|(7,7,7) | больше распределения у центра |
|(5,5,2)| сдвигается к одной из вершин |

<img src="img/dir1.png" width=300>

А если посэмплриуем из распределения Дирихле, то получится примерно вот так:

<img src = "img/dir2.png" siwth=250>

-----


В LDA используется байесовский способ подбора параметров. То есть сначала мы выбраем априорное распределение для параметров, а затем корректируем его наблюдаемыми данными.

В качестве априорных распределений для $\alpha$ и $\beta$ используются распределения Дирихле. Причем используется симметричное распределение 
- $\alpha = \alpha_1 = \alpha_2 = \dots = \alpha_k$ 
- $\beta = \beta_1 = \beta_2 = \dots = \beta_k$

LDA в plate нотации выглядит так: 
<img src="img/lda.png">

Как выглядит генерирующий процесс пошагово?
1. Для каждого документа генерируем своё распределение тематик  $\theta_i \sim Dir(\alpha)$
2. Для каждой тематики генерирем распределение слов $\phi_j \sim Dir(\beta)$
3. Для каждого слова документа выбираем тематику $z_{ij} \sim Multinomial(\theta_i)$
4. Из этой тематики выбираем конкретное слово $w_{ij} \sim Multinomial(\phi_{z_{ij}})$

### Executive Summary
- LDA - классический подход в моделировании тематик документа
- Популярные реализации: *Gensim*
- Perplexity - правдоподобие модели

## Меры качества

### Perplexity 

Perplexity - среднее правдоподобие модели:

$$P=\exp\bigg[ -\frac{\sum{\log{P(w|\alpha,\beta)}}}{\sum{N_d}}\bigg] = \exp^{-E \big[ \log P(w|\alpha, \beta) \big]}$$

где $w$ - слово, $\alpha, \beta$ - наденные параметры модели, $N_d$ - кол-во слов в документе d, всего M документов

Arxiv: <a href="http://dirichlet.net/pdf/wallach09evaluation.pdf">Wallach09</a>

$P \in [0, +\inf)$

- Если имеем полную разделимость по тематикам и средняя $P(w|\alpha,\beta) = 1$, то $P = 0$
- Если всё перемешано и средняя $P(w|\alpha,\beta) = 0$, то $P = 1$

Разумеется, perplexity уменьшается с ростом числа топиков, но можно использовать для подбора других параметров:
<img src="img/perplexity1.png" width = 500>

Здесь например приведена сетка из 2 параметров модели LDA: num_topics и learning_decay

### Coherence

Когерентность тематик - группа метрик, считающих "целостность" топика по словам, его сотавляющим.

Первые метрики были просто расстоянием до плохих topic-word распределений (junk topics). К плохим тематикам можно отнести  те, в которых:
- все слова равноверотяны
- распределение слов в тематике совпадает с распределением во всем корпусе текстов
- тематика распределена равноверотяно по всем документам

Метрики качества топиков можно разделить на
- Extrinsic - которые требуют внешнего датасета для расчета
- Intrinsic - не требуют

Наиболее распространены две метрики: 
- UMass (intrinsic)
- UCI (extrinsic)

Обе сначала считаются для каждой пары слов (pairwise), составляющих тематику, а затем суммируются:

$$Coherence = \sum_{i<j}{score(w_i,w_j)}$$

Arxiv: <a href="http://dirichlet.net/pdf/mimno11optimizing.pdf">Wallach09</a>

UCI - это Pointwise Mutual Information для пары слов w_i и w_j

$$UCI = \log{\frac{P(w_i,w_j)}{P(w_i) \cdot P(w_j)}}$$

UMass

$$UMass = \log{\frac{P(w_i,w_j)+1}{P(w_j)}}$$

В Gensim для расчета когерентности топика используют более продвинутый подход, основанный на алгоритме описанном в <a href="http://svn.aksw.org/papers/2015/WSDM_Topic_Evaluation/public.pdf">WSDM_Topic_Evaluation</a>

<img src="">